### Funzione per il parsing del file i testo generato dal SW di calibrazione della soglia
Riceve in input il nome del file da elaborare e ritorna un oggetto di tipo Dataframe contenente i dati estrapolati dal file di testo.

In [1]:
import numpy as np
import pandas as pd

In [2]:
def parseTxtCalib(file):
    channels = []
    means = []
    sigmas = []
    types = []
    x_lst = []
    y_lst = []
    
    with open (file, 'rt') as myfile:
    
        lines = myfile.readlines()
        
        for i in range(0, len(lines)):
        
        #se la riga contiene la parola 'CHANNEL' significa che siamo nel caso:
        # channel   1 mean 10.30 sigma 0.97
            if ("channel ") in lines[i]:
                
                #se la media è NaN allora non considero il dato e passo al prossimo pixel
                #(siamo nel caso di pixel esclusi dalla calibrazione)
                if lines[i].split()[3] != 'NaN':
                    #salvo il numero del CANALE
                    channels.append(float(lines[i].split()[1]))
                   
                    #salvo la MEDIA
                    means.append(float(lines[i].split()[3]))
                    
                    #salvo le VARIANZE del singolo pixel 
                    sigmas.append(lines[i].split()[5])
                    
                    #stabilisco il TIPO di FE: se non ho i valori dei bit 17-31, allora è RD53B
                    x_points = []
                    y_points = []
                    if lines[i+17].split()[1] == 'NaN':
                        types.append('A')
                    
                        for j in range(1,16+1):
                            y_points.append(float(lines[i+j].split()[0]))
                            x_points.append(j)
                    
                        x_lst.append(x_points)
                        y_lst.append(y_points)
                        
                    else:
                        types.append('B')
                        points = []
                        for j in range(1,32+1):
                            y_points.append(float(lines[i+j].split()[0]))
                            x_points.append(j)
                    
                        x_lst.append(x_points)
                        y_lst.append(y_points)
                
                    #devo salvare i singoli campioni da cui sono state ricavate media e varianza
                    #TO DO
                    
                
                #pixel esclusi dei test
                else:
                    x_points = []
                    y_points = []
                    channels.append(np.nan)
                    means.append(np.nan)
                    sigmas.append(np.nan)
                    types.append(np.nan)
                    
                    for t in range(1,32):
                        #print(t)
                        x_points.append(np.nan)
                        y_points.append(np.nan)
                        
                    x_lst.append(x_points)
                    y_lst.append(y_points)
                       
                
            #per il momento processo solo i dati relativi ai Trimming DAC
            #nel file txt dalla riga successiva a questa ci sono dati che al momento non mi interessa elaborare
            if ("x data 9") in lines[i]: break
                
    #print(lst)
    #creo un oggetto di tipo DataFrame
    return pd.DataFrame(list(zip(channels, types, means, sigmas, x_lst, y_lst)),
                        columns = ['Channel', 'Tipo','Mean','Sigma','HitDots_x', 'HitDots_y'])